In [141]:
!pip install openpyxl
!pip install xgboost
!pip install imblearn
!pip install imbalanced-learn
import sys
!{sys.executable} -m pip install imbalanced-learn
!pip install scikit-learn==1.3.2 imbalanced-learn==0.11.0


Defaulting to user installation because normal site-packages is not writeable


In [142]:
import pandas as pd

skills_df = pd.read_excel("Skills.xlsx")


automation_df = pd.read_csv("automation_data_by_state.csv", encoding='latin1')
technology_df = pd.read_excel("Technology Skills.xlsx")

task_df = pd.read_excel("Task Ratings.xlsx")
work_df=pd.read_excel("Work Activities.xlsx")
occ_df= pd.read_excel("Occupation Data.xlsx")


In [143]:
task_df.columns

Index(['O*NET-SOC Code', 'Title', 'Task ID', 'Task', 'Scale ID', 'Scale Name',
       'Category', 'Data Value', 'N', 'Standard Error', 'Lower CI Bound',
       'Upper CI Bound', 'Recommend Suppress', 'Date', 'Domain Source'],
      dtype='object')

In [144]:
automation_df.columns

Index(['SOC', 'Occupation', 'Probability', 'Alabama', 'Alaska', 'Arizona',
       'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object')

In [145]:
technology_df.columns

Index(['O*NET-SOC Code', 'Title', 'Example', 'Commodity Code',
       'Commodity Title', 'Hot Technology', 'In Demand'],
      dtype='object')

In [146]:
skills_df.columns

Index(['O*NET-SOC Code', 'Title', 'Element ID', 'Element Name', 'Scale ID',
       'Scale Name', 'Data Value', 'N', 'Standard Error', 'Lower CI Bound',
       'Upper CI Bound', 'Recommend Suppress', 'Not Relevant', 'Date',
       'Domain Source'],
      dtype='object')

In [147]:
occ_df.columns

Index(['O*NET-SOC Code', 'Title', 'Description'], dtype='object')

In [148]:

automation_df['SOC'] = automation_df['SOC'].str.strip()


automation_df.rename(columns={'SOC': 'O*NET-SOC Code'}, inplace=True)


In [149]:
work_pivot = work_df.pivot_table(index='O*NET-SOC Code',
                                  columns='Element Name',
                                  values='Data Value')


In [150]:

skills_pivot = skills_df.pivot_table(index='O*NET-SOC Code',
                                     columns='Element Name',
                                     values='Data Value')



In [151]:
technology_df['Title'] = technology_df['Title'].str.strip()


tech_features = pd.crosstab(technology_df['O*NET-SOC Code'], technology_df['Title'])


In [152]:
technology_df

,O*NET-SOC Code,Title,Example,Commodity Code,Commodity Title,Hot Technology,In Demand
0,11-1011.00,Chief Executives,Adobe Acrobat,43232202,Document management software,Y,N
1,11-1011.00,Chief Executives,AdSense Tracker,43232306,Data base user interface and query software,N,N
2,11-1011.00,Chief Executives,Atlassian JIRA,43232201,Content workflow software,Y,N
3,11-1011.00,Chief Executives,Blackbaud The Raiser's Edge,43232303,Customer relationship management CRM software,N,N
4,11-1011.00,Chief Executives,ComputerEase construction accounting software,43231601,Accounting software,N,N
...,...,...,...,...,...,...,...
32676,53-7121.00,"Tank Car, Truck, and Ship Loaders",Linux,43233004,Operating system software,Y,N
32677,53-7121.00,"Tank Car, Truck, and Ship Loaders",Microsoft Excel,43232110,Spreadsheet software,Y,N
32678,53-7121.00,"Tank Car, Truck, and Ship Loaders",Microsoft Office software,43231513,Office suite software,Y,N
32679,53-7121.00,"Tank Car, Truck, and Ship Loaders",SAP software,43231602,Enterprise resource planning ERP software,Y,N


In [153]:

task_importance = task_df[task_df['Scale Name'] == 'Importance']


task_features = task_importance.groupby('O*NET-SOC Code')['Data Value'].mean().reset_index()
task_features.rename(columns={'Data Value': 'Avg Task Importance'}, inplace=True)



In [154]:
print("occ_df codes:", occ_df['O*NET-SOC Code'].nunique())
print("automation_df codes:", automation_df['O*NET-SOC Code'].nunique())
print("Common codes:", set(occ_df['O*NET-SOC Code']).intersection(set(automation_df['O*NET-SOC Code'])))


occ_df codes: 1016
automation_df codes: 702
Common codes: set()


In [155]:
# Standardize the O*NET-SOC Code to remove '.00' or any decimal
# STEP 1: Normalize job codes in occ_df to match automation_df
occ_df['O*NET-SOC Code'] = occ_df['O*NET-SOC Code'].str.strip().str[:7]

# STEP 2: Do the same for other related files
skills_df['O*NET-SOC Code'] = skills_df['O*NET-SOC Code'].str.strip().str[:7]
task_df['O*NET-SOC Code'] = task_df['O*NET-SOC Code'].str.strip().str[:7]
work_df['O*NET-SOC Code'] = work_df['O*NET-SOC Code'].str.strip().str[:7]
# Only do this if automation_df codes are shorter too
automation_df['O*NET-SOC Code'] = automation_df['O*NET-SOC Code'].astype(str).str.strip().str[:7]




In [185]:
final_df.head()

,O*NET-SOC Code,Title,Description,Occupation,Probability,Alabama,Alaska,Arizona,Arkansas,California,...,Repairing and Maintaining Mechanical Equipment,Resolving Conflicts and Negotiating with Others,Scheduling Work and Activities,Selling or Influencing Others,Staffing Organizational Units,Thinking Creatively,Training and Teaching Others,Updating and Using Relevant Knowledge,Working with Computers,Avg Task Importance
0,11-1011,Chief Executives,Determine and formulate policies and provide o...,Chief Executives,0.015,1030,760,5750,2710,31150,...,0.740,5.105,4.12,4.36,4.29,4.385,4.385,4.700,3.600,3.950968
1,11-1011,Chief Executives,Determine and formulate policies and provide o...,Chief Executives,0.015,1030,760,5750,2710,31150,...,0.740,5.105,4.12,4.36,4.29,4.385,4.385,4.700,3.600,3.757222
2,11-1011,Chief Executives,Determine and formulate policies and provide o...,Chief Executives,0.015,1030,760,5750,2710,31150,...,1.095,4.070,4.28,4.59,3.76,4.590,3.925,4.945,3.685,3.950968
3,11-1011,Chief Executives,Determine and formulate policies and provide o...,Chief Executives,0.015,1030,760,5750,2710,31150,...,1.095,4.070,4.28,4.59,3.76,4.590,3.925,4.945,3.685,3.757222
4,11-1011,Chief Executives,Determine and formulate policies and provide o...,Chief Executives,0.015,1030,760,5750,2710,31150,...,0.740,5.105,4.12,4.36,4.29,4.385,4.385,4.700,3.600,3.950968


In [158]:
print("Skills rows:", skills_df.shape[0])
print("Tasks rows:", task_df.shape[0])
print("Work Activities rows:", work_df.shape[0])
print("Tech Skills rows:", technology_df.shape[0])
print("Automation rows:", automation_df.shape[0])
print("Occupation rows:", occ_df.shape[0])


Skills rows: 61530
Tasks rows: 158751
Work Activities rows: 72078
Tech Skills rows: 32681
Automation rows: 702
Occupation rows: 1016


In [186]:
print("Unique job codes in Skills:", skills_df['O*NET-SOC Code'].nunique())
print("Unique job codes in Tasks:", task_df['O*NET-SOC Code'].nunique())
print("Unique job codes in Work:", work_df['O*NET-SOC Code'].nunique())
print("Unique job codes in Tech Skills:", technology_df['O*NET-SOC Code'].nunique())
print("Unique job codes in Automation:", automation_df['O*NET-SOC Code'].nunique())
print("Unique job codes in Occupation:", occ_df['O*NET-SOC Code'].nunique())


Unique job codes in Skills: 763
Unique job codes in Tasks: 763
Unique job codes in Work: 763
Unique job codes in Tech Skills: 923
Unique job codes in Automation: 702
Unique job codes in Occupation: 867


In [160]:
occ_codes = set(occ_df['O*NET-SOC Code'].unique())
auto_codes = set(automation_df['O*NET-SOC Code'].unique())

common_codes = occ_codes.intersection(auto_codes)
print("Common codes between OCC and Automation:", len(common_codes))


Common codes between OCC and Automation: 618


In [161]:
merged_df = occ_df[occ_df['O*NET-SOC Code'].isin(common_codes)].merge(
    automation_df[automation_df['O*NET-SOC Code'].isin(common_codes)],
    on='O*NET-SOC Code',
    how='inner'
)
print("Merged shape:", merged_df.shape)


Merged shape: (721, 56)


In [162]:
base_df = merged_df.copy()
print("Base:", base_df.shape)  # should be (721, ...)


Base: (721, 56)


In [163]:

skills_pivot.reset_index(inplace=True)
work_pivot.reset_index(inplace=True)


def clean_onet_code(code):
    return code.split('.')[0] if isinstance(code, str) and '.' in code else code

skills_pivot['O*NET-SOC Code'] = skills_pivot['O*NET-SOC Code'].apply(clean_onet_code)
work_pivot['O*NET-SOC Code'] = work_pivot['O*NET-SOC Code'].apply(clean_onet_code)
task_features['O*NET-SOC Code'] = task_features['O*NET-SOC Code'].apply(clean_onet_code)
automation_df['O*NET-SOC Code'] = automation_df['O*NET-SOC Code'].apply(clean_onet_code)
occ_df['O*NET-SOC Code'] = occ_df['O*NET-SOC Code'].apply(clean_onet_code)


In [164]:
final_df = merged_df \
    .merge(skills_pivot, on='O*NET-SOC Code', how='left') \
    .merge(work_pivot, on='O*NET-SOC Code', how='left') \
    .merge(task_features, on='O*NET-SOC Code', how='left')

print("Final shape:", final_df.shape)


Final shape: (13350, 133)


In [165]:
column_names_list = list(final_df.columns)

In [166]:
column_names_list

['O*NET-SOC Code',
 'Title',
 'Description',
 'Occupation',
 'Probability',
 'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',
 'Active Learning',
 'Active Listening',
 'Complex Problem Solving',
 'Coordination',
 'Critical Thinking',
 'Equipment Maintenance',
 'Equipment Selection',
 'Installation',
 'Instructing',
 'Judgment and Decision Making',
 'L

In [167]:

columns_to_drop = ['O*NET-SOC Code', 'Title', 'Description', 'Occupation']
df_model = final_df.drop(columns=columns_to_drop)


In [168]:
df_model

,Probability,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Repairing and Maintaining Mechanical Equipment,Resolving Conflicts and Negotiating with Others,Scheduling Work and Activities,Selling or Influencing Others,Staffing Organizational Units,Thinking Creatively,Training and Teaching Others,Updating and Using Relevant Knowledge,Working with Computers,Avg Task Importance
0,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,0.740,5.105,4.120,4.360,4.290,4.385,4.385,4.700,3.600,3.950968
1,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,0.740,5.105,4.120,4.360,4.290,4.385,4.385,4.700,3.600,3.757222
2,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,1.095,4.070,4.280,4.590,3.760,4.590,3.925,4.945,3.685,3.950968
3,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,1.095,4.070,4.280,4.590,3.760,4.590,3.925,4.945,3.685,3.757222
4,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,0.740,5.105,4.120,4.360,4.290,4.385,4.385,4.700,3.600,3.950968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13345,0.910,0,0,0,0,0,130,0,0,0,...,4.725,2.810,2.955,1.745,1.270,3.320,3.975,4.400,3.660,3.985385
13346,0.900,0,0,0,0,1130,190,0,0,0,...,4.430,3.275,3.485,1.935,2.120,3.700,3.095,4.175,2.335,4.162143
13347,0.840,0,0,0,180,220,600,0,0,0,...,4.185,2.695,3.455,2.030,1.800,3.430,3.520,3.605,3.540,4.058571
13348,0.930,2070,210,1980,1530,13850,1710,910,440,590,...,3.070,1.200,1.355,0.525,0.505,1.275,1.455,2.280,1.255,3.975000


In [169]:
df_model.isnull().sum()

Probability                              0
Alabama                                  0
Alaska                                   0
Arizona                                  0
Arkansas                                 0
                                        ..
Thinking Creatively                      1
Training and Teaching Others             1
Updating and Using Relevant Knowledge    1
Working with Computers                   1
Avg Task Importance                      1
Length: 129, dtype: int64

In [170]:
df_model.fillna(0,inplace=True)

In [171]:
def classify_risk(score):
    if score >= 0.66:
        return "High"
    elif score >= 0.33:
        return "Moderate"
    else:
        return "Low"

df_model['Risk_Level'] = df_model['Probability'].apply(classify_risk)

In [172]:

X = df_model.drop(columns=['Probability','Risk_Level']) # Features
y = df_model['Risk_Level']                # Target



In [173]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


In [174]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [175]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [176]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y) 

clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

        High       1.00      1.00      1.00      2330
         Low       1.00      0.99      1.00      2424
    Moderate       1.00      1.00      1.00      2426

    accuracy                           1.00      7180
   macro avg       1.00      1.00      1.00      7180
weighted avg       1.00      1.00      1.00      7180

Confusion Matrix:
 [[2330    0    0]
 [  10 2408    6]
 [   0    0 2426]]


In [182]:
import pickle


with open("Ai_USA_model.pkl", "wb") as f:
    pickle.dump(clf, f)

with open('encoders.pkl', 'wb') as f:
    pickle.dump(le, f)


with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [184]:
df_model


,Probability,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Resolving Conflicts and Negotiating with Others,Scheduling Work and Activities,Selling or Influencing Others,Staffing Organizational Units,Thinking Creatively,Training and Teaching Others,Updating and Using Relevant Knowledge,Working with Computers,Avg Task Importance,Risk_Level
0,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,5.105,4.120,4.360,4.290,4.385,4.385,4.700,3.600,3.950968,Low
1,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,5.105,4.120,4.360,4.290,4.385,4.385,4.700,3.600,3.757222,Low
2,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,4.070,4.280,4.590,3.760,4.590,3.925,4.945,3.685,3.950968,Low
3,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,4.070,4.280,4.590,3.760,4.590,3.925,4.945,3.685,3.757222,Low
4,0.015,1030,760,5750,2710,31150,880,1410,340,2840,...,5.105,4.120,4.360,4.290,4.385,4.385,4.700,3.600,3.950968,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13345,0.910,0,0,0,0,0,130,0,0,0,...,2.810,2.955,1.745,1.270,3.320,3.975,4.400,3.660,3.985385,High
13346,0.900,0,0,0,0,1130,190,0,0,0,...,3.275,3.485,1.935,2.120,3.700,3.095,4.175,2.335,4.162143,High
13347,0.840,0,0,0,180,220,600,0,0,0,...,2.695,3.455,2.030,1.800,3.430,3.520,3.605,3.540,4.058571,High
13348,0.930,2070,210,1980,1530,13850,1710,910,440,590,...,1.200,1.355,0.525,0.505,1.275,1.455,2.280,1.255,3.975000,High


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13350 entries, 0 to 13349
Columns: 130 entries, Probability to Risk_Level
dtypes: float64(78), int64(51), object(1)
memory usage: 13.2+ MB
